In [1]:
from pandas.errors import ParserError
from io import StringIO
import pandas as pd
import numpy as np
import zipfile
import glob

In [2]:
with zipfile.ZipFile('/content/hw2_dataset.zip') as existing_zip:
    existing_zip.extractall()

In [87]:
# sitの時のデータ
data_sit_list = []

# walkの時のデータ
data_walk_list = []

# stepsの時のデータ
data_steps_list = []

dirs = glob.glob('/content/hw2_dataset/*')

# columnsが条件を満たすデータだけを抽出する
# sitのラベルはラベルは0、walkのラベルは1、stepsのデータは2にする
for dir in dirs:

  dir_sit = glob.glob(dir + '/*sit*')
  for sit_file in dir_sit:
    try:
      data_sit = pd.read_csv(sit_file)
      if list(data_sit.columns) == ['xmax', 'xmin', 'xave', 'xvar', 'ymax', 'ymin', 'yave', 'yvar', 'zmax', 'zmin', 'zave', 'zvar']:
        data_sit['label'] = 0
        data_sit_list.append(data_sit)
    except ParserError:
      pass
    except UnicodeDecodeError:
      pass
    
  dir_walk = glob.glob(dir + '/*walk*')
  for walk_file in dir_walk:
    try:
      data_walk = pd.read_csv(walk_file)
      if list(data_walk.columns) == ['xmax', 'xmin', 'xave', 'xvar', 'ymax', 'ymin', 'yave', 'yvar', 'zmax', 'zmin', 'zave', 'zvar']:
        data_walk['label'] = 1
        data_walk_list.append(data_walk)
    except ParserError:
      pass

  dir_steps = glob.glob(dir + '/*steps*')
  for steps_file in dir_walk:
    try:
      data_steps = pd.read_csv(steps_file)
      if list(data_steps.columns) == ['xmax', 'xmin', 'xave', 'xvar', 'ymax', 'ymin', 'yave', 'yvar', 'zmax', 'zmin', 'zave', 'zvar']:
        data_steps['label'] = 2
        data_steps_list.append(data_steps)
    except ParserError:
      pass



In [4]:
# sitのデータをcsv化する
df_sit = pd.concat(data_sit_list, axis=0, sort=True)
df_sit.to_csv("/content/sit.csv",index=False)

# walkのデータをcsv化する
df_walk = pd.concat(data_walk_list, axis=0, sort=True)
df_walk.to_csv("/content/walk.csv",index=False)

# stepsのデータをcsv化する
df_steps = pd.concat(data_steps_list, axis=0, sort=True)
df_steps.to_csv("/content/steps.csv",index=False)

# 学習データ全体のcsvファイルを作成する
df_list = [df_sit, df_walk, df_steps]
df = pd.concat(df_list, axis=0, sort=True)
df.to_csv("/content/data.csv",index=False)

In [88]:
# csvの中を抽出する
df = pd.read_csv("/content/data.csv")

In [78]:
# 欠損したデータの処理をする
df = df.fillna(df.mean())

df_zvar_error = list(df["zvar"])

df_zvar = []
for val in df_zvar_error:
  if val == '0.0294917sit3':
    val = np.nan
  df_zvar.append(val)
  
df["zvar"] = df_zvar
df["zvar"] = df["zvar"].fillna(method="ffill")

In [79]:
#ラベルを数値化
x = df.iloc[:, 1:13]
x = x.astype("float32")
y = df.pop('label')

In [80]:
#学習データと検証データを分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [81]:
import numpy as np
# import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras import layers, Input, Model, losses, optimizers, models
from tensorflow.keras.utils import to_categorical

In [82]:
# ラベルデータをone-hot vectorに変換する
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [83]:
# ニューラルネットワークを定義する
inputs = Input(shape=(12,))

dense = layers.Dense(32, activation="relu")
x = dense(inputs)

dense2 = layers.Dense(64, activation="relu")
x2 = dense2(x)

dense3 = layers.Dense(3, activation="softmax")
outputs = dense3(x)

In [84]:
# モデルを作成する
model = Model(inputs=inputs, outputs=outputs, name="abc_model")

In [85]:
# モデルのパラメータを指定する
model.compile(
    loss=losses.CategoricalCrossentropy(),
    optimizer=optimizers.Adam(lr=1e-1),
    metrics=["accuracy"],
)

# 学習させる
history = model.fit(x_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

# テストデータで正答率を確認する
test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
32/32 [==============================] - 1s 12ms/step - loss: 43.7183 - accuracy: 0.4648 - val_loss: 3.2645 - val_accuracy: 0.6094
Epoch 2/100
32/32 [==============================] - 0s 4ms/step - loss: 1.5167 - accuracy: 0.6409 - val_loss: 1.2604 - val_accuracy: 0.6211
Epoch 3/100
32/32 [==============================] - 0s 4ms/step - loss: 0.9597 - accuracy: 0.6438 - val_loss: 1.4760 - val_accuracy: 0.6289
Epoch 4/100
32/32 [==============================] - 0s 4ms/step - loss: 0.8035 - accuracy: 0.6350 - val_loss: 1.5287 - val_accuracy: 0.6406
Epoch 5/100
32/32 [==============================] - 0s 4ms/step - loss: 0.6870 - accuracy: 0.6399 - val_loss: 3.0695 - val_accuracy: 0.6250
Epoch 6/100
32/32 [==============================] - 0s 4ms/step - loss: 1.3943 - accuracy: 0.6458 - val_loss: 1.0404 - val_accuracy: 0.6445
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 0.6679 - accuracy: 0.6370 - val_loss: 0.9671 - val_accuracy: 0.6445
Epoch 8/100

In [ ]:
model.save("/content/abc_model.h5")
del model
# Recreate the exact same model purely from the file:
model = models.load_model("/content/abc_model.h5")

In [ ]:
!pip install coremltools
!pip install scikit-learn==0.19.2
!pip install tensorflow==2.5.0

     |████████████████████████████████| 1.6 MB 13.1 MB/s 


In [ ]:
import coremltools as ct

# convert to Core ML and check predictions
mlmodel = ct.convert(model)

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 9/9 [00:00<00:00, 6503.92 ops/s]


In [ ]:
mlmodel.save("abc.mlmodel")